# in-path상에서 HTTP Reqeust, HTTP Response 조작
----

개발환경 : OS X

----
1. 송수신되는 패킷에서 TCP Data Change를 할 때에 패킷의 크기 변경은 없도록 한다(Data 크기가 변경이 되면 골치 아픔).
2. 수신된 패킷에서 IP Header, TCP Header, TCP Data의 시작 위치를 구한다.
3. TCP Data 영역(HTTP)에서 특정 패턴에 대해서 값을 변경시킨다.
4. 프로그램의 구동 결과 daum.net에서 "마이클 잭슨"을 검색하였을 경우 기존 Michael이라는 글자가 다른 것(GILBERT)으로 대체되면 성공.

----


in-path에서의 처리를 위하여 **twisted** library를 쓰기로 했다.

**terminal**에서 `pip install twisted` 입력

~~No module named zope.interface에러 발생.~~
-> pip upgrade후 괜찮아짐

In [1]:
import os
from twisted.web import proxy, http
from twisted.internet import endpoints, reactor, protocol
from twisted.python import log
import sys
import getopt

:0: UserWarning: You do not have a working installation of the service_identity module: 'No module named service_identity'.  Please install it from <https://pypi.python.org/pypi/service_identity> and make sure all of its dependencies are satisfied.  Without the service_identity module and a recent enough pyOpenSSL to support it, Twisted can perform only rudimentary TLS client hostname verification.  Many valid certificate/hostname mappings may be rejected.


root인지 check.

In [2]:
#You should be root.
if os.geteuid() != 0:
        sys.exit("Please run as root") 

---
## 1. HTTP Response, Request Message Format

---
## 2. Proxy server 생성


    
* 시스템 환경설정 -> 네트워크 -> 고급 -> 프록시
* hostName -> localhost
* Port# -> 8080

![프록시 설정 방법](img/사진1.png "프록시 설정 방법")

### 1) Web Server에서 오는 Response를 고정적인 페이지로 보내주기
    
[참조 링크1](http://stackoverflow.com/questions/9465236/python-twisted-proxy-and-modifying-content), [참조 링크2](http://sujitpal.blogspot.kr/2010/03/http-debug-proxy-with-twisted.html)
    

In [3]:
class ProxyClient(proxy.ProxyClient):
    """Mangle returned header, content here.

    Use `self.father` methods to modify request directly.
    """
    """
    def handleHeader(self, key, value):
        # change response header here
        log.msg("Header: %s: %s" % (key, value))
        proxy.ProxyClient.handleHeader(self, key, value)
    """
    def handleResponsePart(self, buffer):
        # change response part here
        log.msg("Content: %s" % (buffer[:50],))
        print "Packet Response"
        # make all content upper case
        proxy.ProxyClient.handleResponsePart(self, buffer.upper())

class ProxyClientFactory(proxy.ProxyClientFactory):
    protocol = ProxyClient

class ProxyRequest(proxy.ProxyRequest):
    protocols = dict(http=ProxyClientFactory)

class Proxy(proxy.Proxy):
    requestFactory = ProxyRequest

class ProxyFactory(http.HTTPFactory):
    protocol = Proxy

In [4]:
def shutdown(reason, reactor, stopping=[]):
        """Stop the reactor."""
        if stopping: return
        stopping.append(True)
        if reason:
            log.msg(reason.value)
        reactor.callWhenRunning(reactor.stop)

portstr = "tcp:8080:interface=localhost" 
log.startLogging(sys.stdout)
endpoint = endpoints.serverFromString(reactor, portstr)
d = endpoint.listen(ProxyFactory())
d.addErrback(shutdown, reactor)
reactor.run()

2016-11-05 20:41:43+0900 [-] Log opened.
2016-11-05 20:41:43+0900 [-] ProxyFactory starting on 8080
2016-11-05 20:41:43+0900 [-] Starting factory <__main__.ProxyFactory instance at 0x104b67f80>
2016-11-05 20:41:54+0900 [-] Received SIGINT, shutting down.
2016-11-05 20:41:54+0900 [__main__.ProxyFactory] (TCP Port 8080 Closed)
2016-11-05 20:41:54+0900 [-] Stopping factory <__main__.ProxyFactory instance at 0x104b67f80>
2016-11-05 20:41:54+0900 [-] Main loop terminated.


~~~~
portstr = "tcp:8080:interface=localhost" 
log.startLogging(sys.stdout)
endpoint = endpoints.serverFromString(reactor, portstr)
d = endpoint.listen(ProxyFactory())
d.addErrback(shutdown, reactor)
reactor.run()
~~~~
으로 실행을 하는데, 한번 돌리고 나서는 restart를 하고 돌려야 하는 것 같다. 

그렇지 않으면 **ReactorNotRestartable **에러가 발생한다.


----

### 2) cliet에서 server로 보내는 request packet modify하기

[참조링크](http://stackoverflow.com/questions/9063583/python-twisted-proxy-how-to-intercept-packets)

In [4]:
log.startLogging(sys.stdout)

class MyProxy(proxy.Proxy):
    def dataReceived(self, data):

        #Modify the data here
        print data

        #perform the default functionality on modified data 
        return MyProxy.dataReceived(self, data)

class ProxyFactory(http.HTTPFactory):
    protocol=MyProxy

factory = ProxyFactory()
reactor.listenTCP(8080, factory)
reactor.run()

ReactorNotRestartable: 

In [5]:
class ConsoleWriter():
    """ Write request (on source port) and response (from target host:port) """
    """ to console. Also holds on to the "latest" data received for output  """
    """ into the proxy server's response stream.                            """

    def write(self, data, type):
        if (data):
            lines = data.split("\n")
            prefix = "<" if type == "request" else ">"
            for line in lines:
                sys.stdout.write("%s %s\n" % (prefix, line))
        else:
            sys.stdout.write("No response from server\n")


class DebugHttpClientProtocol(protocol.Protocol):
    """ Client protocol. Writes out the request to the target HTTP server."""
    """ Response is written to stdout on receipt, and back to the server's"""
    """ transport when the client connection is lost.                     """

    def __init__(self, serverTransport):
        self.serverTransport = serverTransport

    def sendMessage(self, data):
        self.transport.write(data)
  
    def dataReceived(self, data):
        self.data = data
        ConsoleWriter().write(data, "response")
        self.transport.loseConnection()

    def connectionLost(self, reason):
        self.serverTransport.write(self.data)
        self.serverTransport.loseConnection()


class DebugHttpServerProtocol(protocol.Protocol):
    """ Server Protocol. Handles data received from client application.   """
    """ Writes the data to console, then creates a proxy client component """
    """ and sends the data through, then terminates the client and server """
    """ connections.                                                      """

    def dataReceived(self, data):
        self.data = data
        ConsoleWriter().write(self.data, "request")
        client = protocol.ClientCreator(reactor, DebugHttpClientProtocol, self.transport)
        d = client.connectTCP(self.factory.targetHost, self.factory.targetPort)
        d.addCallback(self.forwardToClient, client)

    def forwardToClient(self, client, data):
        client.sendMessage(self.data)


class DebugHttpServerFactory(protocol.ServerFactory):
    """ Server Factory. A holder for the protocol and for user-supplied args """

    protocol = DebugHttpServerProtocol

    def __init__(self, targetHost, targetPort):
        self.targetHost = targetHost
        self.targetPort = targetPort



In [8]:
sourcePort, targetHost, targetPort = 80, None, None

# start twisted reactor
reactor.listenTCP(sourcePort,DebugHttpServerFactory(targetHost, int(targetPort)))
reactor.run()

GetoptError: option -f not recognized